In [1]:
!pip install transformers trl peft accelerate datasets bitsandbytes

In [2]:
from huggingface_hub import notebook_login

In [3]:
notebook_login()

In [4]:
from transformers import AutoModelForCausalLM

In [5]:
# model = AutoModelForCausalLM.from_pretrained("TabbyML/SantaCoder-1B")

In [6]:
import torch 
from datasets import load_dataset, Dataset

In [7]:
from trl import SFTTrainer

In [8]:
from accelerate import Accelerator

In [9]:
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments

In [11]:
sft_config={

            "model_ckpt": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "sft-tiny-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5

}

In [12]:
class TrainSFT:

  def __init__(self,data,config):
    self.data=data
    self.config=config

  def prepare_lora_model(self):

    self.lora_config= LoraConfig(r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"])

    self.model= get_peft_model(self.model,self.lora_config)

  def load_model_tokenizer(self):
    self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"])
    self.model.config.use_cache=False
    self.model.config.pretraining_tp=1
    self.model = prepare_model_for_kbit_training(self.model)

    if self.config["use_lora"]:
      self.prepare_lora_model()

    self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
    self.tokenizer.pad_token = self.tokenizer.eos_token

  def set_training_args(self):


    return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )
  def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    peft_config=self.lora_config,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )
        else:
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )

  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"])
    self.tokenizer.save_pretrained(self.config["output_dir"])



In [13]:
def create_data():
    data = load_dataset("tatsu-lab/alpaca", split="train")
    data_df = data.to_pandas()
    data_df = data_df[:700]
    data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
    data = Dataset.from_pandas(data_df)
    return data

data = create_data()

In [14]:
train_sft = TrainSFT(data, sft_config)
train_sft.train_and_save_model()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
None


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=Tru

Step,Training Loss
100,2.180000
200,1.765700


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always